TODO: Redo as a lasso type fit from sklearn. Divide the sample into 3 sets (train test validate) for validating the size of the hyperparameter for the lasso. You'll need to take the multipolyfit for dividing your z's into the appropriate basis.

In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sympy import pprint

multipolyfit from https://github.com/mrocklin/multipolyfit/blob/master/multipolyfit/core.py

In [4]:
from numpy import linalg, zeros, ones, hstack, asarray
import itertools

def basis_vector(n, i):
    """ Return an array like [0, 0, ..., 1, ..., 0, 0]
    >>> from multipolyfit.core import basis_vector
    >>> basis_vector(3, 1)
    array([0, 1, 0])
    >>> basis_vector(5, 4)
    array([0, 0, 0, 0, 1])
    """
    x = zeros(n, dtype=int)
    x[i] = 1
    return x

def as_tall(x):
    """ Turns a row vector into a column vector """
    return x.reshape(x.shape + (1,))

def multipolyfit(xs, y, deg, full=False, model_out=False, powers_out=False):
    """
    Least squares multivariate polynomial fit
    Fit a polynomial like ``y = a**2 + 3a - 2ab + 4b**2 - 1``
    with many covariates a, b, c, ...
    Parameters
    ----------
    xs : array_like, shape (M, k)
         x-coordinates of the k covariates over the M sample points
    y :  array_like, shape(M,)
         y-coordinates of the sample points.
    deg : int
         Degree o fthe fitting polynomial
    model_out : bool (defaults to True)
         If True return a callable function
         If False return an array of coefficients
    powers_out : bool (defaults to False)
         Returns the meaning of each of the coefficients in the form of an
         iterator that gives the powers over the inputs and 1
         For example if xs corresponds to the covariates a,b,c then the array
         [1, 2, 1, 0] corresponds to 1**1 * a**2 * b**1 * c**0
    See Also
    --------
        numpy.polyfit
    """
    y = asarray(y).squeeze()
    rows = y.shape[0]
    xs = asarray(xs)
    num_covariates = xs.shape[1]
    xs = hstack((ones((xs.shape[0], 1), dtype=xs.dtype) , xs))

    generators = [basis_vector(num_covariates+1, i)
                            for i in range(num_covariates+1)]

    # All combinations of degrees
    powers = map(sum, itertools.combinations_with_replacement(generators, deg))

    # Raise data to specified degree pattern, stack in order
    A = hstack(asarray([as_tall((xs**p).prod(1)) for p in powers]))

    outs = linalg.lstsq(A, y)
    beta = outs[0]
    

    if model_out:
        return mk_model(beta, powers)

    if powers_out:
        return beta, powers
    return beta

def mk_model(beta, powers):
    """ Create a callable python function out of beta/powers from multipolyfit
    This function is callable from within multipolyfit using the model_out flag
    """
    # Create a function that takes in many x values
    # and returns an approximate y value
    def model(*args):
        num_covariates = len(powers[0]) - 1
        if len(args)!=(num_covariates):
            raise ValueError("Expected %d inputs"%num_covariates)
        xs = asarray((1,) + args)
        return sum([coeff * (xs**p).prod()
                             for p, coeff in zip(powers, beta)])
    return model


def mk_sympy_function(beta, powers):
    from sympy import symbols, Add, Mul, S
    num_covariates = len(powers[0]) - 1
    xs = (S.One,) + symbols('z4:%d'%(num_covariates+4))
    return Add(*[coeff * Mul(*[x**deg for x, deg in zip(xs, power)])
                        for power, coeff in zip(powers, beta)])

In [5]:
from WavefrontPSF.donutengine import Zernike_to_Pixel_Interpolator
from WavefrontPSF.psf_evaluator import Moment_Evaluator
PSF_Drawer = Zernike_to_Pixel_Interpolator()
PSF_Evaluator = Moment_Evaluator()

def evaluate_psf(data):
    stamps, data = PSF_Drawer(data)
    evaluated_psfs = PSF_Evaluator(stamps)
    # this is sick:
    combined_df = evaluated_psfs.combine_first(data)

    return combined_df

In [6]:
rzero = 0.14
x = 0
y = 0

Ndim = 7
Nsample = 100
zmax = 0.2

# zernikes = np.meshgrid(*[np.linspace(-zmax, zmax, Nsample) for i in xrange(Ndim)])
# # z4 = zernikes[0], z5 = zernikes[1], etc
# N = zernikes[0].size

zernikes = np.random.random(size=(Ndim, Nsample)) * (zmax + zmax) - zmax

data = {'rzero': np.ones(N) * rzero,
        'x': np.ones(N) * x,
        'y': np.ones(N) * y}

x_keys = []
for zi, zernike in enumerate(zernikes):
    zkey = 'z{0}'.format(zi + 4)
    data[zkey] = zernike.flatten()
    x_keys.append(zkey)
df = pd.DataFrame(data)
df = evaluate_psf(df)

y_keys = ['flux', 'e0prime', 'e0', 'e1', 'e2', 'delta1', 'delta2', 'zeta1', 'zeta2']

In [8]:
def power_selector(zi, zj, powers, beta):
    powersum = np.sum(powers[0])
    powers_desired = np.zeros(len(powers[0]), dtype=np.int)
    powers_desired[zi - 3] += 1
    powers_desired[zj - 3] += 1
    powers_desired[0] = powersum - np.sum(powers_desired)
    ith = np.array([np.all(power == powers_desired) for power in powers])
    return beta[ith]

In [9]:
yith = 2
deg = 2

xs = df[x_keys].values
ys = df[y_keys].values
y = ys[:, yith]  # for e0

beta, powers = multipolyfit(xs=xs, y=y, deg=deg, model_out=False, powers_out=True)

beta_ith = power_selector(4, 4, powers, beta)
beta *= 24. / beta_ith

pprint(mk_sympy_function(beta, powers))

        2                                                              2      
25.2⋅z₁₀  + 0.1⋅z₁₀⋅z₅ - 0.1⋅z₁₀⋅z₆ - 0.2⋅z₁₀⋅z₈ - 0.1⋅z₁₀⋅z₉ + 24.0⋅z₄  + 0.6

                            2          2                                  2   
⋅z₄⋅z₆ - 0.1⋅z₄⋅z₉ + 12.4⋅z₅  + 12.3⋅z₆  + 0.1⋅z₆⋅z₈ - 0.1⋅z₆⋅z₉ + 36.6⋅z₇  - 

                   2          2       
1.0⋅z₇⋅z₉ + 35.3⋅z₈  + 25.2⋅z₉  + 58.0


In [11]:
from IPython.html.widgets import interact

def func(zi_txt='4', zj_txt='4', const_txt='1.', yith=2, deg=1, threshold_txt='0'):
    threshold = float(threshold_txt)
    y = ys[:, yith]
    beta, powers = multipolyfit(xs=xs, y=y, deg=4, model_out=False, powers_out=True)
    zi = int(zi_txt)
    zj = int(zj_txt)
    const = float(const_txt)
    beta = beta * const
    beta = np.where(np.abs(beta) > threshold, beta, 0)
    beta = np.around(beta, deg)
    print(y_keys[yith], zi, zj, power_selector(zi, zj, powers, beta))
    pprint(mk_sympy_function(beta, powers))
interact(func, deg=(0,5), yith=(0, 8))

e0 4 4 [ 0.09545515]
       2                                               2   2         2   2    
0.1⋅z₁₀  + 0.1⋅z₁₀⋅z₄⋅z₅⋅z₇ - 0.1⋅z₁₀⋅z₄⋅z₆⋅z₈ - 0.1⋅z₄ ⋅z₇  - 0.1⋅z₄ ⋅z₈  + 0

     2                                                   2                    
.1⋅z₄  - 0.1⋅z₄⋅z₅⋅z₇⋅z₈ - 0.1⋅z₄⋅z₅⋅z₈⋅z₉ + 0.1⋅z₄⋅z₆⋅z₇  - 0.1⋅z₄⋅z₆⋅z₇⋅z₉ +

       2         2         2      
 0.1⋅z₇  + 0.1⋅z₈  + 0.1⋅z₉  + 0.2


<function __main__.func>

In [140]:
power_selector(6, 10, powers, beta)

array([-0.02106377])